# <center>**Notebook update_tables()**</center>

O propósito desta função é aprimorar o processo de webscraping das outras funções. Ao fazer requests para extrair o HTML das tabelas no site UFCstats, há um risco de exceder o limite de requisições, o que pode resultar no bloqueio do IP do usuário, impedindo futuros requests. Para contornar essa situação e otimizar o tempo, vamos criar uma função chamada "update_table". Essa função irá determinar se é necessário atualizar a tabela ou se as funções podem simplesmente usar as tabelas previamente salvas, evitando assim o problema de ultrapassar o limite de requisições permitidas.

### Bibliotecas utilizadas

In [3]:
# Importando bibliotecas
# Webscraping
import requests
from bs4 import BeautifulSoup

# Manipulação de dados
import pandas as pd
import numpy as np

## Funcionamento

A função update_tables() será executada dentro das outras funções presentes no arquivo ufc_scrap.py. Primeiro, ela verificará se as tabelas armazenadas no diretório "dataframes" estão atualizadas de acordo com o site UFCstats. Para fazer essa verificação, ela acessará a última linha da tabela df_events() e comparará se o nome do evento ali registrado coincide com o último evento publicado no site.

O output dessa função será um valor booleano: True se o dataframe estiver atualizado e False se estiver desatualizado, indicando a necessidade de executar o webscraping novamente.

In [47]:
# Criando função

def update_tables():
    # Acessando ultimo evento registrado na base de dados
    df_events = pd.read_csv("../dataframes/df_events.csv")

    # Obtendo nome do ultimo evento
    event_database = df_events["event"][1]

    # Link de acesso da pagina de eventos
    URL = "http://www.ufcstats.com/statistics/events/completed?page=all"

    # Realizando request para site
    response = requests.get(URL)
    response.raise_for_status()

    # Obtendo pagina completa em html
    soup = BeautifulSoup(response.content, "html.parser")

    # Encontrando tabela
    table = soup.find("table", class_="b-statistics__table-events")

    # Encontrando linhas
    rows = table.find_next("a", class_="b-link b-link_style_black")

    # Obtendo o nome do evento
    event_newest = rows.text
    
    # Tratando o nome do evento antes da comparação
    event_newest = event_newest.strip()

    if event_database == event_newest:
        return True
    else:
        return False
    
update_tables()

False

Assim temos a função que faz a conferencia se o dataframe esta ou não atualizado, retornando uma resposta booleana.

## Aplicação

Como exemplo de aplicação iremos utilizar função get_events() para demonstrar.

A função note_update é executada antes de todo o script. Se ela retornar True, o dataframe salvo já é retornado, evitando a necessidade de realizar o webscraping.

Nota: A localização dos dataframes no script ufc_scrap será diferente, pois esse script está em outro diretório.

In [48]:
# Aqui está a função get_events() completa
def get_events(bruto=False):
    """
    Função que retorna o data frame com todos os eventos realizados pelo UFC e armazenados no site UFCstats.
    """

    # Executando verificação de webscraping
    if update_tables():
        df_events = pd.read_csv("../dataframes/df_events.csv")
        df_events = df_events.drop(columns=["Unnamed: 0"])
        print("Deu certo")
        return df_events


    # Link de acesso para pagina
    URL = "http://www.ufcstats.com/statistics/events/completed?page=all"


    # Realizando o request para o site 
    response = requests.get(URL)
    response.raise_for_status()

    # Obtendo pagina completa em html
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Encontrando a tabela com a classe especificada
    table = soup.find("table", class_="b-statistics__table-events")

    # Transformando tabela de html para dataframe pandas ou retornando erro se não encontrar a tabela
    if table:
        # Extraindo cabeçalho
        header = [th.text.strip() for th in table.find('thead').find_all('th')]

        # Extraindo as linhas
        rows = table.find('tbody').find_all('tr')
        table_data = [[col.text.strip() for col in row.find_all('td')] for row in rows]

        # Convertendo em dataframe
        df_bruto = pd.DataFrame(table_data[1:], columns=header) #Obs: Não considera a primeira linha

    else:
        return print("Não foi possível encontrar a tabela desejada na página.")


    if bruto:
        return df_bruto

    """
    A parte do código abaixo é dedicata ao tratamento do df_bruto para os parametros pré estabelecidos no arquivo note_get_events.ipynb.
    """

    # Criando o dataframe final
    df_final = df_bruto.copy()

    # Ajustando coluna evento
    df_final['event'] = df_final['Name/date'].str.split("\n")
    df_final['event'] = df_final['event'].apply(lambda x: x[0])

    # Ajustando data
    df_final['date'] = df_final['Name/date'].apply(lambda x: x.split("\n")[-1].strip())
    df_final['date'] = pd.to_datetime(df_final['date'])

    # Retirando coluna Name/date
    df_final = df_final.drop(columns=["Name/date"])

    # Criando colunas de cidade, estado e pais
    # Cidade
    df_final['city'] = df_final['Location'].apply(lambda x: x.split(",")[0].strip())

    # Estado
    df_final['state'] = df_final['Location'].apply(lambda x: x.split(",")[1].strip() if len(x.split(",")) == 3 else np.nan)

    # Pais
    df_final['country'] = df_final['Location'].apply(lambda x: x.split(",")[-1].strip())

    # Removendo a coluna Location
    df_final = df_final.drop(columns=["Location"])


    """
    A parte do código a seguir, cria uma primary key para a tabela de evetos, para mais detalhes olhar o arquivo documentado note_get_events.ipynb
    """

    # Criando range com o index invertido
    inverted_index = df_final.index[::-1] + 1

    # Criando a chave primaria no formato EVEXXXX
    df_final['event_id'] = ["EVE" + f"{index:04}" for index in inverted_index]

    """
    Como ultimo tratamento, as colunas serão reorganizadas
    """
    df_final = df_final[["event_id", "event", "date","city", "state", "country"]]

    # Salvando dataframe atualizado na pasta dataframes
    df_final.to_csv("../dataframes/df_events.csv")
    print("deu errado")

    return df_final

In [50]:
# Testando função
df_events = get_events()

df_events

Deu certo


,event_id,event,date,city,state,country
0,EVE0664,UFC Fight Night: Fiziev vs. Gamrot,2023-09-23,Las Vegas,Nevada,USA
1,EVE0663,UFC Fight Night: Grasso vs. Shevchenko 2,2023-09-16,Las Vegas,Nevada,USA
2,EVE0662,UFC 293: Adesanya vs. Strickland,2023-09-09,Sydney,New South Wales,Australia
3,EVE0661,UFC Fight Night: Gane vs. Spivac,2023-09-02,Paris,Ile-de-France,France
4,EVE0660,UFC Fight Night: Holloway vs. The Korean Zombie,2023-08-26,Kallang,NaN,Singapore
...,...,...,...,...,...,...
659,EVE0005,UFC 6: Clash of the Titans,1995-07-14,Casper,Wyoming,USA
660,EVE0004,UFC 5: The Return of the Beast,1995-04-07,Charlotte,North Carolina,USA
661,EVE0003,UFC 4: Revenge of the Warriors,1994-12-16,Tulsa,Oklahoma,USA
662,EVE0002,UFC 3: The American Dream,1994-09-09,Charlotte,North Carolina,USA
